# 1. Normalização do dataset: Ocorrências aeronáuticas

# 2. Membros (nome e número de matrícula)
- Izabela Garcia (2022076332)
- Isabella Carvalho (2022037523)
- Gabriel Vitor Gonçalves (2022076308)
- João Marcos Rezende (2022076340)

In [1]:
import csv
import pandas as pd
import sqlite3

# 3. Descrição dos dados (qual a URL? qual o domínio? como os dados foram processados?)
## URL da base de dados

https://dados.gov.br/dados/conjuntos-dados/ocorrencias-aeronauticas

## Domínio dos dados

Este conjunto de dados contém informações sobre **ocorrências aeronáuticas**, incluindo acidentes e incidentes graves registrados pela Força Aérea Brasileira (FAB) e a ANAC. As ocorrências envolvem dados como a aeronave, o tipo de operação, o local, descrição do ocorrido, e as lesões causadas aos tripulantes, passageiros e terceiros.

### Objetivo do Dataset

O objetivo desse dataset é fornecer informações para análise e melhoria da **segurança operacional** na aviação, permitindo a investigação de incidentes e a identificação de padrões de risco. Ele é utilizado para monitorar a segurança, apoiar investigações e desenvolver estratégias para prevenção de acidentes.

## Pré-processamento dos dados
### 1. Padronização de Valores Nulos
Identificamos diferentes representações de valores ausentes no dataset, como `""`, `"null"`, `"Indeterminado"`, `"-"` e outras. Todas foram padronizadas para `NULL`. Para as colunas relacionadas a lesões (numéricas), valores nulos foram interpretados como `0`.

### 2. Remoção de Colunas Redundantes
As colunas `Operador` e `Operador_Normalizado` apresentavam informações equivalentes. Optamos por manter apenas `Operador_Normalizado` (após renomeá-la para `Operador`), simplificando o dataset.

### 3. Renomeação de Colunas
Para melhorar a legibilidade e adequação ao dicionário de dados, renomeamos colunas do dataset:
- `Historico` → `Descricao`
- `Operador_Padronizado` → `Operador`
- `Tipo_de_Ocorrencia` → `Tipo_ADREP`
- `Descricao_do_Tipo` → `Tipo_descricao`
- `Tipo_ICAO` → `Cod_ICAO`

### 4. Remoção de Linhas
#### a. Linhas com Valores Nulos em Colunas Importantes
Excluímos linhas com valores nulos em colunas críticas como `Matricula`, `Operacao`, `Regiao` e `Descricao`, pois essas informações são fundamentais para o estudo ou atuarão como IDs no banco de dados.

#### b. Duplicatas
A coluna `Numero_da_Ocorrencia` será usada como chave primária no banco de dados. Linhas duplicadas foram reduzidas a uma única instância, mantendo o número único.

#### c. Valores Desconhecidos
Linhas onde o campo `Operador` continha valores como `"OPERADOR DESCONHECIDO"` ou `"DESCONHECIDO"` foram removidas, pois não podem ser agrupadas semanticamente.

In [2]:
source = 'V_OCORRENCIA_AMPLA.csv'
df = pd.read_csv(source, delimiter=';',quotechar='"', header=1)

# padronizando nulos
df.replace({'-': None, 'null': None, 'Indeterminado': None, '': None, 'XXXX': None, 'ZZZZ': None, '***': None, '*** ': None, '**** ': None, '****': None, '*****': None, '###!': None}, inplace=True)
df['Lesoes_Fatais_Tripulantes'] = df['Lesoes_Fatais_Tripulantes'].fillna('0')
df['Lesoes_Fatais_Passageiros'] = df['Lesoes_Fatais_Passageiros'].fillna('0')
df['Lesoes_Fatais_Terceiros'] = df['Lesoes_Fatais_Terceiros'].fillna('0')
df['Lesoes_Graves_Tripulantes'] = df['Lesoes_Graves_Tripulantes'].fillna('0')
df['Lesoes_Graves_Passageiros'] = df['Lesoes_Graves_Passageiros'].fillna('0')
df['Lesoes_Graves_Terceiros'] = df['Lesoes_Graves_Terceiros'].fillna('0')
df['Lesoes_Leves_Tripulantes'] = df['Lesoes_Graves_Tripulantes'].fillna('0')
df['Lesoes_Leves_Passageiros'] = df['Lesoes_Graves_Passageiros'].fillna('0')
df['Lesoes_Leves_Terceiros'] = df['Lesoes_Graves_Terceiros'].fillna('0')
df['Ilesos_Tripulantes'] = df['Ilesos_Tripulantes'].fillna('0')
df['Ilesos_Passageiros'] = df['Ilesos_Passageiros'].fillna('0')
df['Lesoes_Desconhecidas_Tripulantes'] = df['Lesoes_Desconhecidas_Tripulantes'].fillna('0')
df['Lesoes_Desconhecidas_Passageiros'] = df['Lesoes_Desconhecidas_Passageiros'].fillna('0')
df['Lesoes_Desconhecidas_Terceiros'] = df['Lesoes_Desconhecidas_Terceiros'].fillna('0')

# deletar coluna do operador
df.drop("Operador", axis=1, inplace=True)

# renomear colunas
df.columns = df.columns.str.strip()
df.rename(columns={'Historico': 'Descricao'}, inplace=True)
df.rename(columns={'Operador_Padronizado': 'Operador'}, inplace=True)
df.rename(columns={'Tipo_de_Ocorrencia': 'Tipo_ADREP'}, inplace=True)
df.rename(columns={'Descricao_do_Tipo':'Tipo_descricao'}, inplace=True)
df.rename(columns={'Tipo_ICAO':'Cod_ICAO'}, inplace=True)
print(f'Tamanho original do dataset: {len(df)}') # tamanho original

# remoção de linhas com descrição nula
df = df.dropna(subset=['Descricao'])

# remoção de linhas com operador "DESCONHECIDO", "OPERADOR DESCONHECIDO" ou nulo
df = df.query(' Operador != "DESCONHECIDO" and Operador != "OPERADOR DESCONHECIDO" ')
df = df.dropna(subset=['Operador'])

# remoção de linhas com operação nula
df = df.dropna(subset=['Operacao'])

# remoção de linhas com aeronave com cod_ICAO nulo
df = df.dropna(subset=['Matricula'])

# remoção de linhas com Região nulos
df = df.dropna(subset=['Regiao'])

# remoção de duplicatas com mesmo número de ocorrência
df = df.drop_duplicates(subset=['Numero_da_Ocorrencia'], keep='first')

# checagem de ID
if df['Numero_da_Ocorrencia'].isna().sum() == 0 and df['Numero_da_Ocorrencia'].nunique() == len(df):
    print("Numero_da_Ocorrencia pode ser utilizado como ID")
else:
    print("Numero_da_Ocorrencia não pode ser utilizado como ID")

print(f'Tamanho do dataset após remoções: {len(df)}\n') # tamanho após remoções

print(df.info())

Tamanho original do dataset: 5021
Numero_da_Ocorrencia pode ser utilizado como ID
Tamanho do dataset após remoções: 3468

<class 'pandas.core.frame.DataFrame'>
Index: 3468 entries, 0 to 5020
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Numero_da_Ocorrencia              3468 non-null   int64  
 1   Numero_da_Ficha                   3435 non-null   object 
 2   Operador                          3468 non-null   object 
 3   Classificacao_da_Ocorrencia       3468 non-null   object 
 4   Data_da_Ocorrencia                3468 non-null   object 
 5   Hora_da_Ocorrencia                2851 non-null   object 
 6   Municipio                         2894 non-null   object 
 7   UF                                3468 non-null   object 
 8   Regiao                            3468 non-null   object 
 9   Tipo_descricao                    3398 non-null   object 
 10  ICAO           

### Tabela pré-processada

In [3]:
# Salvar o DataFrame em um novo arquivo CSV
output_csv = "ocorrencias_limpo.csv"
df.to_csv(output_csv, index=False, sep=';', encoding='utf-8')

### Criando o banco

In [4]:
src = 'ocorrencias_limpo.csv'

conn = sqlite3.connect('ocorrencias.db')
cur = conn.cursor()

cur.execute('PRAGMA synchronous = OFF')
cur.execute('PRAGMA journal_mode = MEMORY')

BATCH_SIZE = 1000

cur.execute('BEGIN TRANSACTION')

with open(src, 'r', newline='', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=';')
    header = next(reader)

    columns = [f'"{column}"' for column in header]
    create = f'CREATE TABLE IF NOT EXISTS Ocorrencias ({", ".join(columns)})'
    cur.execute(create)

    placeholders = ', '.join(['?'] * len(header))
    insert = f'INSERT INTO Ocorrencias VALUES ({placeholders})'

    batch = []
    for row in reader:
        row = [None if value == '' else value for value in row]
        batch.append(row)
        if len(batch) == BATCH_SIZE:
            cur.executemany(insert, batch)
            batch = []

    if batch:
        cur.executemany(insert, batch)

update = f'UPDATE Ocorrencias SET ' + ','.join([f'{column} = NULLIF({column}, "")' for column in columns])
cur.execute(update)

conn.commit()

cur.execute('PRAGMA synchronous = FULL')
cur.execute('PRAGMA journal_mode = DELETE')

## Compreensão dos dados

In [5]:
def fetch(query, conn, formatted=True):
    cur = conn.cursor()
    cur.execute(query)
    rs = cur.fetchall()

    columns = [desc[0] for desc in cur.description]

    return pd.DataFrame(rs, columns=columns) if formatted else rs

def desc(table, conn):
    cur = conn.cursor()
    cur.execute(f'PRAGMA table_info("{table}")')
    columns = [row[1] for row in cur.fetchall()]

    return columns

def info(table, conn):
    df1 = fetch(f'PRAGMA table_info("{table}")', conn)
    columns = desc(table, conn)

    counts = ', '.join([f'COUNT(*) AS "{column}"' for column in columns])
    df2 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df2.columns = ['count']

    counts = ', '.join([f'COUNT("{column}") AS "{column}"' for column in columns])
    df3 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df3.columns = ['notnull count']

    counts = ', '.join([f'COUNT(DISTINCT "{column}") AS "{column}"' for column in columns])
    df4 = fetch(f'SELECT {counts} FROM "{table}"', conn).transpose()
    df4.columns = ['unique count']

    return df1.merge(df2, left_on='name', right_index=True) \
            .merge(df3, left_on='name', right_index=True) \
            .merge(df4, left_on='name', right_index=True)

info('Ocorrencias', conn)

,cid,name,type,notnull,dflt_value,pk,count,notnull count,unique count
0,0,Numero_da_Ocorrencia,,0,None,0,3468,3468,3468
1,1,Numero_da_Ficha,,0,None,0,3468,3435,3192
2,2,Operador,,0,None,0,3468,3468,2513
3,3,Classificacao_da_Ocorrencia,,0,None,0,3468,3468,2
4,4,Data_da_Ocorrencia,,0,None,0,3468,3468,2584
5,5,Hora_da_Ocorrencia,,0,None,0,3468,2851,681
6,6,Municipio,,0,None,0,3468,2894,1288
7,7,UF,,0,None,0,3468,3468,27
8,8,Regiao,,0,None,0,3468,3468,5
9,9,Tipo_descricao,,0,None,0,3468,3398,33


In [6]:
cur.close()
conn.close()

## Normalização das Tabelas
### 1. **Tabela `ocorrencia`**
Contém informações principais sobre as ocorrências aeronáuticas, como o número da ocorrência, classificação, data, hora e descrição.
- **Chave primária**: `Numero_da_Ocorrencia`.

### 2. **Tabela `local`**
Armazena a localização das ocorrências, com a sigla da unidade federativa (`UF`) e a região.
- **Chave primária**: `UF`.

### 3. **Tabela `aeronave`**
Contém informações sobre as aeronaves envolvidas nas ocorrências, incluindo matrícula, código ICAO, modelo, capacidade, fabricante e categoria da aeronave.
- **Chave primária**: `Matricula`.

### 4. **Tabela `aerodromo`**
Armazena dados sobre os aeródromos, como o código ICAO e o tipo de aeródromo.
- **Chave primária**: `ICAO`.

### 5. **Tabela `operador`**
Armazena o nome do operador responsável pela aeronave durante a ocorrência. Cada operador é associado a uma ocorrência específica.
- **Chave primária**: `Numero_da_Ocorrencia`.

### 6. **Tabela `lesao`**
Esta tabela detalha as lesões associadas a cada ocorrência, com informações sobre a pessoa afetada, o tipo de lesão e a quantidade de pessoas. As lesões são categorizadas em tipos como fatais, graves e leves, e distinguem entre tripulantes, passageiros e terceiros.
- **Chaves primárias compostas**: `Numero_da_Ocorrencia`, `Pessoa_afetada`, `Tipo_Lesao`.

### 7. **Tabela `local_ocorrencia`**
Armazena as coordenadas geográficas, o município, número e UF das ocorrências.
- **Chave primária composta**: `UF`, `Numero_da_Ocorrencia`.

### 8. **Tabela `aeronave_ocorrencia`**
Relaciona as aeronaves envolvidas nas ocorrências, com a matrícula da aeronave e o número da ocorrência.
- **Chave primária composta**: `Matricula`, `Numero_da_Ocorrencia`.

### 9. **Tabela `lesoes_ocorrencia`**
Armazena os tipos de lesões associadas às ocorrências, separadas por pessoa afetada e tipo de lesão.
- **Chaves primárias compostas**: `Numero_da_Ocorrencia`, `Pessoa_afetada`, `Tipo_Lesao`.

### 10. **Tabela `voo`**
Relaciona as ocorrências com os voos, especificando os aeródromos de origem e destino de cada voo envolvido na ocorrência.
- **Chave primária composta**: `Aerodromo_de_Origem`, `Numero_da_Ocorrencia`.

### 11. **Tabela `operacao`**
Armazena informações sobre o tipo e fase da operação no momento da ocorrência.
- **Chave primária**: `Numero_da_Ocorrencia`.

In [7]:
conn = sqlite3.connect('ocorrencias.db')
cur = conn.cursor()

# Carregar a tabela principal em um DataFrame
df = pd.read_sql_query("SELECT * FROM Ocorrencias", conn)

# Criar as tabelas com chaves primárias e estrangeiras usando SQL

# Tabela `ocorrencia`
conn.execute("""
CREATE TABLE IF NOT EXISTS ocorrencia (
    Numero_da_Ocorrencia INTEGER PRIMARY KEY,
    Numero_da_Ficha INTEGER,
    Classificacao_da_Ocorrencia TEXT,
    Data_da_Ocorrencia TEXT,
    Hora_da_Ocorrencia TEXT,
    Descricao TEXT,
    Tipo_ADREP TEXT,
    Tipo_descricao TEXT,
    Danos_a_Aeronave TEXT,
    PSSO TEXT
);
""")

# Inserir dados na tabela `ocorrencia`
ocorrencia = df[[
    "Numero_da_Ocorrencia", "Numero_da_Ficha", "Classificacao_da_Ocorrencia",
    "Data_da_Ocorrencia", "Hora_da_Ocorrencia", "Descricao", "Tipo_ADREP",
    "Tipo_descricao", "Danos_a_Aeronave", "PSSO"
]]
ocorrencia.to_sql("ocorrencia", conn, if_exists="append", index=False)

# Tabela `local`
conn.execute("""
CREATE TABLE IF NOT EXISTS local (
    UF TEXT PRIMARY KEY,
    Regiao TEXT
);
""")

local = df[["UF", "Regiao"]].drop_duplicates()
local.to_sql("local", conn, if_exists="append", index=False)

# Tabela `aeronave`
conn.execute("""
CREATE TABLE IF NOT EXISTS aeronave (
    Matricula TEXT PRIMARY KEY,
    Cod_ICAO TEXT,
    Modelo TEXT,
    CLS TEXT,
    PMD INTEGER,
    Numero_de_Assentos INTEGER,
    Nome_do_Fabricante TEXT,
    Categoria_da_Aeronave TEXT
);
""")

aeronave = df[[
    "Matricula", "Cod_ICAO", "Modelo", "CLS", "PMD",
    "Numero_de_Assentos", "Nome_do_Fabricante", "Categoria_da_Aeronave"
]].drop_duplicates(subset=['Matricula'])
aeronave.to_sql("aeronave", conn, if_exists="append", index=False)

# Tabela `aerodromo`
conn.execute("""
CREATE TABLE IF NOT EXISTS aerodromo (
    ICAO TEXT PRIMARY KEY,
    Tipo_de_Aerodromo TEXT
);
""")

aerodromo = df[["ICAO", "Tipo_de_Aerodromo"]].drop_duplicates(subset=['ICAO'])
aerodromo.to_sql("aerodromo", conn, if_exists="append", index=False)
cur.execute("DELETE FROM aerodromo WHERE ICAO = 'None'")

# Tabela `operador`
conn.execute("""
CREATE TABLE IF NOT EXISTS operador (
    Numero_da_Ocorrencia INTEGER PRIMARY KEY,
    Nome_do_Operador TEXT
);
""")

operador = df[["Numero_da_Ocorrencia", "Operador"]].rename(columns={"Operador": "Nome_do_Operador"})
operador.to_sql("operador", conn, if_exists="append", index=False)

# Tabela `Lesao`
conn.execute("""
CREATE TABLE IF NOT EXISTS lesao (
    Numero_da_Ocorrencia INTEGER,
    Pessoa_afetada TEXT,
    Tipo_Lesao TEXT,
    Quantidade INTEGER,
    PRIMARY KEY (Numero_da_Ocorrencia, Pessoa_afetada, Tipo_Lesao)

);
""")

lesoes_cols = [
    "Lesoes_Fatais_Tripulantes", "Lesoes_Fatais_Passageiros", "Lesoes_Fatais_Terceiros",
    "Lesoes_Graves_Tripulantes", "Lesoes_Graves_Passageiros", "Lesoes_Graves_Terceiros",
    "Lesoes_Leves_Tripulantes", "Lesoes_Leves_Passageiros", "Lesoes_Leves_Terceiros",
    "Ilesos_Tripulantes", "Ilesos_Passageiros", "Lesoes_Desconhecidas_Tripulantes",
    "Lesoes_Desconhecidas_Passageiros", "Lesoes_Desconhecidas_Terceiros"
]

lesoes = []

# Iterar sobre as colunas de lesões no DataFrame
for col in lesoes_cols:
    # Separar o tipo de lesão e a pessoa afetada
    aux = col.split("_")
    if aux[0] == 'Lesoes':
        aux.pop(0)
    tipo_lesao = aux[0].capitalize()
    pessoa_afetada = aux[1].capitalize()
    # Criar uma entrada para cada linha
    for _, row in df.iterrows():
        quantidade = row[col]
        lesoes.append({
            "Numero_da_Ocorrencia": row["Numero_da_Ocorrencia"],
            "Pessoa_afetada": pessoa_afetada,
            "Tipo_Lesao": tipo_lesao,
            "Quantidade": quantidade
        })

lesoes_df = pd.DataFrame(lesoes)
lesoes_df.to_sql("lesao", conn, if_exists="append", index=False)

# Tabela `local_ocorrencia`
conn.execute("""
CREATE TABLE IF NOT EXISTS local_ocorrencia (
    Latitude REAL,
    Longitude REAL,
    Municipio TEXT,
    UF TEXT,
    Numero_da_Ocorrencia INTEGER,
    PRIMARY KEY (UF, Numero_da_Ocorrencia)
);
""")

localOcorrencia = df[[
    "Latitude", "Longitude", "Municipio", "UF", "Numero_da_Ocorrencia"]]
localOcorrencia.to_sql("local_ocorrencia", conn, if_exists="append", index=False)

# Tabela `aeronave_ocorrencia`
conn.execute("""
CREATE TABLE IF NOT EXISTS aeronave_ocorrencia (
    Matricula TEXT,
    Numero_da_Ocorrencia INTEGER,
    PRIMARY KEY (Matricula, Numero_da_Ocorrencia)
);
""")

aeronaveOcorrencia = df[["Matricula", "Numero_da_Ocorrencia"]]
aeronaveOcorrencia.to_sql("aeronave_ocorrencia", conn, if_exists="append", index=False)

# Tabela `lesoes_ocorrencia`
conn.execute("""
CREATE TABLE IF NOT EXISTS lesoes_ocorrencia (
    Numero_da_Ocorrencia INTEGER,
    Pessoa_afetada TEXT,
    Tipo_Lesao TEXT,
    PRIMARY KEY (Numero_da_Ocorrencia, Pessoa_afetada, Tipo_Lesao)
);
""")

# Inserir os dados de `lesoes_df` na tabela `lesoes_ocorrencia`
for _, row in lesoes_df.iterrows():
    conn.execute("""
    INSERT OR IGNORE INTO lesoes_ocorrencia (Numero_da_Ocorrencia, Pessoa_afetada, Tipo_Lesao)
    VALUES (?, ?, ?)
    """, (row["Numero_da_Ocorrencia"], row["Pessoa_afetada"], row["Tipo_Lesao"]))

conn.commit()

# Tabela `voo`
conn.execute("""
CREATE TABLE IF NOT EXISTS voo (
    Aerodromo_de_Origem TEXT,
    Aerodromo_de_Destino TEXT,
    Numero_da_Ocorrencia INTEGER,
    PRIMARY KEY (Aerodromo_de_Origem, Numero_da_Ocorrencia)
);
""")

voo = df[["Aerodromo_de_Origem", "Aerodromo_de_Destino", "Numero_da_Ocorrencia"]]
voo.to_sql("voo", conn, if_exists="append", index=False)

# Tabela `operacao`
conn.execute("""
CREATE TABLE IF NOT EXISTS operacao (
    Tipo_da_Operacao TEXT,
    Fase_da_Operacao TEXT,
    Numero_da_Ocorrencia INTEGER,
    PRIMARY KEY (Numero_da_Ocorrencia)
);
""")

operacao = df[[
     "Operacao", "Fase_da_Operacao", "Numero_da_Ocorrencia"
]].rename(columns={"Operacao": "Tipo_da_Operacao"})
operacao.to_sql("operacao", conn, if_exists="append", index=False)

# Listar tabelas criadas
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
print("Tabelas criadas:", tables["name"].tolist())

# Fechar conexão
conn.close()


Tabelas criadas: ['Ocorrencias', 'ocorrencia', 'local', 'aeronave', 'aerodromo', 'operador', 'lesao', 'local_ocorrencia', 'aeronave_ocorrencia', 'lesoes_ocorrencia', 'voo', 'operacao']


## Verificação das tabelas criadas

In [8]:
# Conectar ao banco de dados
conn = sqlite3.connect("ocorrencias.db")

tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table' AND name != 'Ocorrencias'",
    conn
)

for table_name in tables["name"]:
    print(f"\nConteúdo da tabela `{table_name}`:")
    table_content = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    print(table_content)



Conteúdo da tabela `ocorrencia`:
      Numero_da_Ocorrencia Numero_da_Ficha Classificacao_da_Ocorrencia  \
0                        1       0902/2013                    Acidente   
1                        7       1651/2013                    Acidente   
2                        8       1652/2013             Incidente Grave   
3                        9       1647/2013                    Acidente   
4                       19       0843/2013                    Acidente   
...                    ...             ...                         ...   
3463                 46055    202441807619                    Acidente   
3464                 46056    202461806837                    Acidente   
3465                 46057    202406542059                    Acidente   
3466                 46058    202422192270                    Acidente   
3467                 46134    202461801440                    Acidente   

     Data_da_Ocorrencia Hora_da_Ocorrencia  \
0            2013-04-25        

# 4. Diagrama ER

<img src="https://raw.githubusercontent.com/belatavaresp/TP2---IBD/refs/heads/main/Diagrama%20ER.drawio%20(1).svg" alt="" width="1100">

# 5. Diagrama relacional

- **Ocorrência**
  - Atributos: `n_ocorrência`, `n_ficha`, `classificação`, `data`, `hora`, `descrição`, `tipo_ADREP`, `tipo_descrição`, `danos_aeronave`, `PSSO`
  - Chave primária: `n_ocorrência`

- **Local**
  - Atributos: `UF`, `região`
  - Chave primária: `UF`

- **Aeronave**
  - Atributos: `matrícula`, `cod_ICAO`, `modelo`, `CLS`, `PMD`, `n_assentos`, `fabricante`, `categoria_aeronave`
  - Chave primária: `matricula`

- **Aeródromo**
  - Atributos: `ICAO`, `tipo_aeródromo`
  - Chave primária: `ICAO`

- **Operador**
  - Atributos: `n_ocorrência`, `nome_operador`
  - Chave estrangeira: `n_ocorrência -> Ocorrência`

- **Lesão**
  - Atributos: `pessoa_afetada`, `tipo_lesão`, `quantidade`
  - Chave primária composta: `pessoa_afetada`, `tipo_lesão`

- **local_ocorrência**
  - Atributos: `n_ocorrência`, `UF`, `latitude`, `longitude`, `município`
  - Chaves estrangeiras:
    - `n_ocorrência -> Ocorrência`
    - `UF -> Local`

- **aeronave_ocorrência**
  - Atributos: `n_ocorrência`, `matrícula`
  - Chaves estrangeiras:
    - `n_ocorrência -> Ocorrência`
    - `matrícula -> Aeronave`

- **lesões_ocorrência**
  - Atributos: `n_ocorrência`, `pessoa_afetada`, `tipo_lesão`
  - Chaves estrangeiras:
    - `n_ocorrência -> Ocorrência`
    - `pessoa_afetada, tipo_lesão -> Lesão`

- **voo**
  - Atributos: `n_ocorrência`, `Origem`, `Destino`
  - Chaves estrangeiras:
    - `n_ocorrência -> Ocorrência`
    - `Origem, Destino -> Aeródromo`

- **operação**
  - Atributos: `n_ocorrência`, `Operador`, `Tipo`, `Fase`
  - Chaves estrangeiras:
    - `n_ocorrência -> Ocorrência`
    - `Operador -> Aeródromo`

# 6. Consultas

## 6.1 Duas consultas envolvendo seleção e projeção

### 6.1.1 Consulta 1
Descrição e data das ocorrências de 2024

In [9]:
query = """
SELECT Descricao, Data_Da_Ocorrencia
FROM ocorrencia
WHERE Data_da_Ocorrencia > '2023-12-31'
"""

df = pd.read_sql_query(query, conn)
df

,Descricao,Data_da_Ocorrencia
0,"A aeronave decolou do Aeródromo Bauru (SBBU), ...",2024-01-06
1,A aeronave decolou do heliponto Escarpa (SJXD)...,2024-01-02
2,A aeronave decolou da área de pouso para uso a...,2024-01-12
3,A aeronave decolou do Anel Rodoviário de Belo ...,2024-01-08
4,A aeronave decolou da área de pouso para uso a...,2024-01-17
...,...,...
109,A aeronave decolou do aeródromo Santa Colomba ...,2024-06-11
110,"A aeronave decolou do aeródromo Bauru (SBBU), ...",2024-06-10
111,A aeronave decolou do aeródromo Jair Feliciano...,2024-06-12
112,A aeronave tinha previsão de decolagem do aeró...,2024-06-14


### 6.1.2 Consulta 2
Número das ocorrências e fase das operações que são do tipo Voo Privado

In [10]:
query = """
SELECT Numero_da_Ocorrencia, Fase_da_Operacao
FROM operacao
WHERE Tipo_da_Operacao == 'Voo Privado'
"""

df = pd.read_sql_query(query, conn)
df

,Numero_da_Ocorrencia,Fase_da_Operacao
0,1,Em rota
1,7,Decolagem
2,9,Em rota
3,19,Decolagem
4,21,Táxi
...,...,...
1627,45893,Aproximação
1628,45912,Subida
1629,46056,Decolagem
1630,46058,Decolagem


## 6.2 Três consultas envolvendo junção de duas relações

### 6.2.1 Consulta 3
Fases e número das ocorrências que são classificadas como incidentes graves

In [11]:
query = """
SELECT O.Numero_da_Ocorrencia, OP.Fase_da_Operacao
FROM ocorrencia AS O
NATURAL JOIN operacao as OP
WHERE O.Classificacao_da_Ocorrencia == 'Incidente Grave'
"""

df = pd.read_sql_query(query, conn)
df

,Numero_da_Ocorrencia,Fase_da_Operacao
0,8,Pouso
1,21,Táxi
2,32,Em rota
3,45,Corrida após pouso
4,50,Táxi
...,...,...
1017,45792,Pouso
1018,45878,Pouso
1019,45907,Pouso
1020,45912,Subida


### 6.2.2 Consulta 4
Código ICAO dos aeródromos de origem, data e classificação das ocorrências do ano de 2024

In [12]:
query = """
SELECT V.Aerodromo_de_Origem, O.Data_da_Ocorrencia, O.Classificacao_da_Ocorrencia
FROM ocorrencia AS O
NATURAL JOIN voo as V
WHERE O.Data_da_Ocorrencia > '2023-12-31' AND V.Aerodromo_de_Origem != 'None'
"""

df = pd.read_sql_query(query, conn)
df

,Aerodromo_de_Origem,Data_da_Ocorrencia,Classificacao_da_Ocorrencia
0,NI,2024-04-09,Acidente
1,SBBE,2024-02-21,Incidente Grave
2,SBBE,2024-04-01,Acidente
3,SBBH,2024-03-06,Acidente
4,SBBH,2024-05-04,Incidente Grave
...,...,...,...
65,SWMV,2024-01-21,Acidente
66,SWPY,2024-04-29,Acidente
67,SWTS,2024-04-24,Acidente
68,SWXV,2024-02-08,Incidente Grave


### 6.2.3 Consulta 5
Descrição do tipo, data e classificação das ocorrências com destino ou origem no Aeródromo de Confins (SBCF)

In [13]:
query = """
SELECT O.tipo_descricao, O.Data_da_Ocorrencia, O.Classificacao_da_Ocorrencia
FROM ocorrencia AS O
NATURAL JOIN voo AS V
WHERE V.Aerodromo_de_Origem == 'SBCF' OR V.Aerodromo_de_Destino == 'SBCF'
"""

df = pd.read_sql_query(query, conn)
df

,Tipo_descricao,Data_da_Ocorrencia,Classificacao_da_Ocorrencia
0,EXCURSÃO DE PISTA,2014-09-22,Incidente Grave
1,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,2018-07-16,Acidente
2,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,2018-04-06,Incidente Grave
3,FALHA OU MAU FUNCIONAMENTO DO MOTOR,2017-02-22,Incidente Grave
4,FOGO/FUMAÇA,2017-02-21,Incidente Grave
5,OUTROS,2014-09-25,Acidente


## 6.3 Três consultas envolvendo junção de três ou mais relações

### 6.3.1 Consulta 6
Encontrar o código ICAO dos aeródromos de origem e a descrição de tipo das ocorrências que não tiveram passageiros ilesos

In [14]:
# essa consulta demora em torno de 20s
query = """
SELECT DISTINCT V.Aerodromo_de_Origem, O.tipo_descricao
FROM voo AS V
NATURAL JOIN ocorrencia AS O
NATURAL JOIN lesoes_ocorrencia AS LO
JOIN lesao AS L ON LO.Pessoa_afetada = L.Pessoa_afetada AND LO.Tipo_Lesao = L.Tipo_Lesao
WHERE NOT EXISTS (
    SELECT 1
    FROM lesoes_ocorrencia AS LO_sub
    JOIN lesao AS L_sub ON LO_sub.Pessoa_afetada = L_sub.Pessoa_afetada AND LO_sub.Tipo_Lesao = L_sub.Tipo_Lesao
    WHERE LO_sub.Numero_da_Ocorrencia = O.Numero_da_Ocorrencia
      AND L_sub.Pessoa_afetada = 'Passageiros'
      AND L_sub.Tipo_Lesao = 'Ileso'
) AND V.Aerodromo_de_Origem != 'None';
"""

df = pd.read_sql_query(query, conn)
df

,Aerodromo_de_Origem,Tipo_descricao
0,**NI,EXCURSÃO DE PISTA
1,9PMM,COLISÃO NO SOLO
2,APUA,PERDA DE CONTROLE EM VOO
3,APUA,FALHA OU MAU FUNCIONAMENTO DO MOTOR
4,APUA,OPERAÇÃO A BAIXA ALTITUDE
...,...,...
1518,SWZD,COMBUSTÍVEL
1519,SWZQ,EXCURSÃO DE PISTA
1520,SWZY,FALHA OU MAU FUNCIONAMENTO DO MOTOR
1521,WNK,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...


### 6.3.2 Consulta 7
Encontrar o ICAO dos Aeródromos de destino e de origem com ocorrências na fase de decolagem

In [15]:
query = """
SELECT V.Aerodromo_de_Destino, V.Aerodromo_de_Origem
FROM ocorrencias AS O
NATURAL JOIN voo AS V
NATURAL JOIN operacao as OP
WHERE OP.Fase_da_Operacao == 'Decolagem'
"""

df = pd.read_sql_query(query, conn)
df

,Aerodromo_de_Destino,Aerodromo_de_Origem
0,SWBB,SWBB
1,SSAK,SSAK
2,SSNG,SSNG
3,SBPR,SBPR
4,SSOK,SBBI
...,...,...
239,SBBH,SBBH
240,SBBU,SBBU
241,SDOV,SBIT
242,SNQV,SBPR


### 6.3.3 Consulta 8
Quantas ocorrências do tipo “acidente” possui cada aeródromo de destino ou origem do tipo Público, ordenadas pelo de maior quantidade para o de menor quantidade

In [16]:
query = """
SELECT COUNT(O.Numero_da_Ocorrencia) AS qtd_ocorrencias, A.ICAO
FROM ocorrencias AS O
NATURAL JOIN voo AS V
JOIN aerodromo AS A ON V.Aerodromo_de_Origem == A.ICAO OR V.Aerodromo_de_Destino == A.ICAO
WHERE  O.Classificacao_da_Ocorrencia == 'Acidente' AND A.Tipo_de_Aerodromo == 'Público'
GROUP BY A.ICAO
ORDER BY qtd_ocorrencias DESC;
"""

df = pd.read_sql_query(query, conn)
df

,qtd_ocorrencias,ICAO
0,46,SBMT
1,22,SBJD
2,21,SBJR
3,20,SBBI
4,19,SBPR
...,...,...
134,1,SBJI
135,1,SBIZ
136,1,SBHT
137,1,SBCZ


## 6.4 Duas consultas envolvendo agregação sobre junção de duas ou mais relações

### 6.4.1 Consulta 9
Quantidades acidentes e incidentes graves por região do Brasil

In [17]:
query = """
SELECT
    L.Regiao,
    COUNT(CASE WHEN O.Classificacao_da_Ocorrencia = 'Incidente Grave' THEN 1 END) AS qtd_incidentes_graves,
    COUNT(CASE WHEN O.Classificacao_da_Ocorrencia = 'Acidente' THEN 1 END) AS qtd_acidentes
FROM ocorrencia AS O
	NATURAL JOIN local_ocorrencia
	NATURAL JOIN local AS L
GROUP BY L.Regiao
"""

df = pd.read_sql_query(query, conn)
df

,Regiao,qtd_incidentes_graves,qtd_acidentes
0,Centro-Oeste,225,551
1,Nordeste,137,239
2,Norte,132,376
3,Sudeste,355,796
4,Sul,173,484


### 6.4.2 Consulta 10
Quantidade de ocorrências por fabricante, listadas em ordem decrescente

In [18]:
query = """
SELECT
    A.Nome_do_Fabricante,
    COUNT(AO.Numero_da_Ocorrencia) AS qtd_ocorrencias
FROM aeronave AS A
NATURAL JOIN aeronave_ocorrencia AS AO
NATURAL JOIN ocorrencia AS O
WHERE A.Nome_do_Fabricante != 'None'
GROUP BY A.Nome_do_Fabricante
ORDER BY qtd_ocorrencias DESC;
"""

df = pd.read_sql_query(query, conn)
df

,Nome_do_Fabricante,qtd_ocorrencias
0,CESSNA AIRCRAFT,635
1,NEIVA,605
2,EMBRAER,297
3,PIPER AIRCRAFT,291
4,BEECH AIRCRAFT,176
...,...,...
336,AEROBRAVO IND. AERONAUTICA LTDA,1
337,AEROALCOOL TECNOLOGIA LTDA,1
338,AERO BRAVO,1
339,ADILSON CARMO E SILVA,1


In [19]:
conn.close()

# 7. Autoavaliação dos membros

### Contribuições Individuais

- **Izabela Tavares**:
  - Realizou a limpeza e o pré-processamento dos dados originais.
  - Organizou o notebook utilizado no trabalho.

- **João Marcos Rezende**:
  - Responsável pela criação e normalização das tabelas no banco de dados.

- **Gabriel Vitor Gonçalves**:
  - Criou os diagramas Entidade-Relacionamento (ER) e Relacional.
  - Desenvolveu a aplicação Web com o Streamlit

- **Isabella Araújo**:
  - Desenvolveu e realizou as consultas ao banco de dados final.

### Contribuições em Equipe

- Todos os integrantes participaram da idealização e definição das tabelas e seus respectivos relacionamentos.
- Cada membro desempenhou suas responsabilidades com dedicação e contribuiu produtivamente para o sucesso do trabalho.

## EXTRA: Aplicação Web via	Streamlit para	visualização	interativa	dos	dados	modelados	por	meio	das	consultas	produzidas.

### Link: https://appibd-eg2qb8s3mmk2hxqbqsd2q3.streamlit.app/